# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: Ratas.py 🐁

- Nombre de alumno 1: Geraldyn Pérez
- Nombre de alumno 2: Diego Rojas

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Geral37/MDS7202.git)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [1]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [4]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

`escriba su respuesta acá`

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [15]:
total_rewards = [] # recompensas

# Repetir la simulación 5000 veces
for _ in range(5000):
    # Reiniciar el entorno
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Elegir una acción aleatoria (el espacio de acción tiene 2 dimensiones: "acción 0" o "acción 1")
        action = env.action_space.sample()  # Esto selecciona una acción aleatoria

        # Tomar la acción en el entorno
        observation, reward, done, _, _ = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    # Guardar la recompensa total del episodio
    total_rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
total_rewards = np.array(total_rewards)

# Calcular el promedio y desviación estándar
average_reward = np.mean(total_rewards)
std_deviation = np.std(total_rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de recompensas: -0.4002
Desviación estándar de las recompensas: 0.893554676558743


Es mala perfomance porque la estrategia siempre entrega pérdida en valor esperado. El juego aleatorio estaría cargado en contra del jugador.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [16]:
from stable_baselines3 import A2C

# Crear el modelo A2C con una red neuronal de 64 neuronas en una capa oculta
model = A2C("MlpPolicy", env, verbose=1)

# Entrenar el modelo (10000 pasos de entrenamiento)
model.learn(total_timesteps=10000)

# Evaluar el modelo en 10 episodios
for episode in range(10):
    obs, info = env.reset()  # Reiniciar el entorno
    done = False
    total_reward = 0
    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model.predict(obs)

        # Ejecutar la acción en el entorno
        obs, reward, done, truncated, info = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    print(f"Episodio {episode + 1} - Recompensa Total: {total_reward}")

# Guardar el modelo entrenado (opcional)
model.save("a2c_blackjack")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.09     |
|    ep_rew_mean        | -0.3     |
| time/                 |          |
|    fps                | 137      |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.247   |
|    explained_variance | 0.536    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.0453  |
|    value_loss         | 0.566    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.04     |
|    ep_rew_mean        | -0.27    |
| time/                 |          |
|    fps                | 199      |
|    iterations         | 200      |
|    time_elapsed       | 5        |
|    total_timesteps    | 1000     |
| train/                |          |
|

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [19]:
# Cargar el modelo A2C entrenado (si lo tienes guardado)
#model = A2C.load("a2c_blackjack")

# Lista para almacenar las recompensas
total_rewards = []

# Repetir la simulación 5000 veces
for _ in range(5000):
    # Reiniciar el entorno
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model.predict(observation)

        # Tomar la acción en el entorno
        observation, reward, done, truncated, _ = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    # Guardar la recompensa total del episodio
    total_rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
total_rewards = np.array(total_rewards)

# Calcular el promedio y desviación estándar
average_reward = np.mean(total_rewards)
std_deviation = np.std(total_rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de recompensas: -0.1886
Desviación estándar de las recompensas: 0.9582432050372182


Mantiene baja performance pero mejora respecto al baseline. Se obtiene un promedio de pérdidas menor.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [21]:
env.reset()

(array([20,  2,  0]), {})

In [22]:
# Función para obtener la acción del agente dado un estado
def obtener_accion(state):
    # Usamos el modelo entrenado para predecir la acción en base al estado
    action, _states = model.predict(state)
    return action

# Primer escenario: Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene un as
# Para simular este escenario, creamos un estado artificialmente. Suponemos que el estado es un vector de numpy.
# Nota: La representación del estado dependerá de cómo esté estructurado el entorno. Para fines de este ejercicio,
# asumimos que el estado es un array que contiene [suma_agente, suma_dealer, tiene_as].
# Este es un ejemplo simplificado.

estado_escenario_1 = np.array([6, 7, 0])  # Suma del agente es 6, dealer muestra 7, agente no tiene un as
accion_1 = obtener_accion(estado_escenario_1)

# Segundo escenario: Suma de cartas del agente es 19, dealer muestra un 3, agente tiene un as
estado_escenario_2 = np.array([19, 3, 1])  # Suma del agente es 19, dealer muestra 3, agente tiene un as
accion_2 = obtener_accion(estado_escenario_2)

# Imprimir las acciones en ambos escenarios
print(f"Acción en el escenario 1 (Agente suma 6, Dealer muestra 7, sin as): {accion_1}")
print(f"Acción en el escenario 2 (Agente suma 19, Dealer muestra 3, con as): {accion_2}")

Acción en el escenario 1 (Agente suma 6, Dealer muestra 7, sin as): 0
Acción en el escenario 2 (Agente suma 19, Dealer muestra 3, con as): 0


### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [24]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [25]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

`escriba su respuesta acá`

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [32]:
# Lista para almacenar las recompensas totales de cada episodio
rewards = []

# Ejecutar la simulación 10 veces con acciones aleatorias
for episode in range(10):
    obs, info = env.reset()  # Reiniciar el entorno
    done = False
    total_reward = 0

    while not done:
        # Seleccionar una acción aleatoria
        action = env.action_space.sample()

        # Tomar la acción en el entorno
        obs, reward, done, truncated, info = env.step(action)

        # Acumular la recompensa total
        total_reward += reward

    # Guardar la recompensa total del episodio
    rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
rewards = np.array(rewards)

# Calcular el promedio y desviación estándar de las recompensas
average_reward = np.mean(rewards)
std_deviation = np.std(rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de recompensas: -176.59248950091904
Desviación estándar de las recompensas: 113.9127838304885


Al igual que el baseline del punto anterior, en promedio se obtiene pérdidas al seguir esta estrategia.

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [33]:
from stable_baselines3 import TD3

# Crear el modelo A2C con una red neuronal de 64 neuronas en una capa oculta
model2 = TD3("MlpPolicy", env, verbose=1)

# Entrenar el modelo (10000 pasos de entrenamiento)
model2.learn(total_timesteps=10000)

# Evaluar el modelo en 10 episodios
for episode in range(10):
    obs, info = env.reset()  # Reiniciar el entorno
    done = False
    total_reward = 0
    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model2.predict(obs)

        # Ejecutar la acción en el entorno
        obs, reward, done, truncated, info = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    print(f"Episodio {episode + 1} - Recompensa Total: {total_reward}")

# Guardar el modelo entrenado (opcional)
model2.save("td2_LunarLander")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 74.8     |
|    ep_rew_mean     | -428     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 222      |
|    time_elapsed    | 1        |
|    total_timesteps | 299      |
| train/             |          |
|    actor_loss      | 8.1      |
|    critic_loss     | 601      |
|    learning_rate   | 0.001    |
|    n_updates       | 198      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 111      |
|    ep_rew_mean     | -528     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 153      |
|    time_elapsed    | 5        |
|    total_timesteps | 890      |
| train/             |          |
|    actor_loss      | 7.12     |
|    critic_loss     

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [34]:
# Cargar el modelo A2C entrenado (si lo tienes guardado)
#model2 = TD3.load("td2_LunarLander")

# Lista para almacenar las recompensas
total_rewards = []

# Repetir la simulación 5000 veces
for _ in range(5000):
    # Reiniciar el entorno
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model2.predict(observation)

        # Tomar la acción en el entorno
        observation, reward, done, truncated, _ = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    # Guardar la recompensa total del episodio
    total_rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
total_rewards = np.array(total_rewards)

# Calcular el promedio y desviación estándar
average_reward = np.mean(total_rewards)
std_deviation = np.std(total_rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Promedio de recompensas: -190.66586398909587
Desviación estándar de las recompensas: 128.5650290282243


#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
%pip install --upgrade --quiet PyPDF2

In [ ]:
import PyPDF2

doc_paths = [] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )